In [ ]:
from simulator import Simulator
from network import Network

from network import servers_csv_to_dict, pings_csv_to_dict

network = Network.from_dicts(pings_csv_to_dict("./../pings.csv"), servers_csv_to_dict("./../servers.csv"), fraction = 1)

## GossipSub

In [ ]:
from gossip_algorithm import GossipSub
from attacker import create_random_attackers, UniformEstimator

simulator = Simulator(network=network, gossip_algorithm=GossipSub(network, fanout = 8))
all_nodes = [node.node_id for node in simulator.network.nodes]

stretches_gs = []
attacks_gs = []

for i in range(100):
    print(f"{i+1} / 100") if i % 10 == 0 else None
    simulator.setup()
    lst_attackers = create_random_attackers(UniformEstimator, all_nodes, simulator.first_source.node_id, 0.1, num_attackers = 10)
    stretch, attack  = simulator.run(attackers = lst_attackers, msg_receival_limit = 1)
    stretches_gs.append(stretch)
    attacks_gs.append(attack)

## Hierarchical

In [ ]:
from gossip_algorithm import HierarchicalIntraCobraWalkInterBernoulliWithVoronoi
from attacker import create_random_attackers, UniformEstimator

simulator = Simulator(network=network, gossip_algorithm=HierarchicalIntraCobraWalkInterBernoulliWithVoronoi(network, inter_cluster_probability = 0.7, intra_cobra_walk_rho = 0.7, fanout_inter = 3, num_clusters = 8))
all_nodes = [node.node_id for node in simulator.network.nodes]

stretches_h = []
attacks_h = []

for i in range(100):
    print(f"{i+1} / 100") if i % 10 == 0 else None
    simulator.setup()
    lst_attackers = create_random_attackers(UniformEstimator, all_nodes, simulator.first_source.node_id, 0.1, num_attackers = 10)
    stretch, attack  = simulator.run(attackers = lst_attackers, msg_receival_limit = 60)
    stretches_h.append(stretch)
    attacks_h.append(attack)

In [ ]:
# Histograms
import matplotlib.pyplot as plt
import numpy as np

all_stretches_gs = [stretch.values for stretch in stretches_gs]
all_stretches_gs = [item for sublist in all_stretches_gs for item in sublist]
all_stretches_h = [stretch.values for stretch in stretches_h]
all_stretches_h = [item for sublist in all_stretches_h for item in sublist]
all_attacks_gs = [attack.values for attack in attacks_gs]
all_attacks_gs = [item for sublist in all_attacks_gs for item in sublist]
all_attacks_h = [attack.values for attack in attacks_h]
all_attacks_h = [item for sublist in all_attacks_h for item in sublist]

plt.hist(all_stretches_gs, bins = 20, range = (0, 10), color = 'r', alpha = 0.5, label = "GossipSub")
plt.hist(all_stretches_h, bins = 20, range = (0, 10), color = 'b', alpha = 0.5, label = "Hierarchical")
plt.legend(loc='upper right')
plt.title("Histogram of stretches")
plt.show()

# attack success rate
success_rate_gs = np.mean(all_attacks_gs)
success_rate_h = np.mean(all_attacks_h)
plt.bar(["GossipSub", "Hierarchical"], [success_rate_gs, success_rate_h], color = ['r', 'b'])
plt.title("Attack success rate")
plt.show()

